<a href="https://colab.research.google.com/github/ogunjosam/ISIMIP_data/blob/main/projected_widespread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()
!sudo apt install gcc-9
!sudo apt install libstdc++6
!conda install -y -c conda-forge esmpy xesmf python=3.7

# Input key
url = 'url: https://cds.climate.copernicus.eu/api/v2'
key = 'key: 7930:d2b86b26-b915-42d8-b4b4-d55b4a8983ec'
with open('/root/.cdsapirc', 'w') as f:
    f.write('\n'.join([url, key]))
with open('/root/.cdsapirc') as f:
    print(f.read())

!pip install cdsapi 
import cdsapi
#!python -m pip install "xarray[io]"

import os
import shutil
import zipfile
import os, sys, os.path
from zipfile import ZipFile
import shutil
import zipfile
import netCDF4
try:
  import xesmf as xe 
  import xarray as xr
  import pandas as pd
  import numpy as np
except OSError:
  try:
    import xarray as xr
    import pandas as pd
    import numpy as np
    import xesmf as xe
  except:
    print('try something else')
else:
  import xarray as xr
  import pandas as pd
  import numpy as np
  import xesmf as xe


!pip3 -y uninstall cffi
!pip install --upgrade cffi==1.15.0

!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy

url: https://cds.climate.copernicus.eu/api/v2
key: 7930:d2b86b26-b915-42d8-b4b4-d55b4a8983ec
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Usage:   
  pip3 <command> [options]

no such option: -y
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 427 kB 4.9 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.14.5
    Uninstalling cffi-1.14.5:
      Successfully uninstalled cffi-1.14.5
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libproj-dev is already the newest version (4.9.3-2).
libproj-dev set to manually installed.
proj-data is already the newest version (4.9.3-2).
proj-data set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be i

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.8 MB 2.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 40.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 20.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 20.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 12.1 MB 31.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 2.0 MB 39.7 MB/s 
  Created w

In [3]:
source = ['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR','CAMS-CSM1-0','CanESM5','CESM2','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1',
'EC-Earth3-CC','EC-Earth3-Veg-LR','FGOALS-g3','GFDL-ESM4','HadGEM3-GC31-LL','IITM-ESM','INM-CM4-8','INM-CM5-0','IPSL-CM6A-LR',
          'KACE-1-0-G','KIOST-ESM','MIROC6','MIROC-ES2L','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-LM','NorESM2-MM','UKESM1-0-LL']

In [79]:
def era5_extractor(experiment,ii):
  llat = 4.59
  ulat = 23.1
  llon = -19.96
  rlon = 15.46

  c = cdsapi.Client()
  c.retrieve('projections-cmip6',
                {
                'format': 'zip',
                'temporal_resolution': 'daily',
                'experiment': experiment,
                'level': 'single_levels',
                'variable': 'precipitation',
                'date': '1981-01-01/2100-12-31',
                'area': [
                        23.1, -19.96, 15.46,
                        4.59,
                        ],
                'model': ii,
                  },
              'temp.zip')    
      
  with zipfile.ZipFile('temp.zip',"r") as zip_ref:
    listOfFileNames = zip_ref.namelist()
    for fileName in listOfFileNames:
      if fileName.endswith('.nc'):
        zip_ref.extract(fileName)
        break

  pp = xr.open_dataset(fileName, chunks={"latitude": 10, "longitude": 10})
  if 'time_bnds' in list(pp.variables):
	  pp = pp.drop_vars('time_bnds')
      
  if 'lon_bnds' in list(pp.variables):
	  pp = pp.drop_vars('lon_bnds')
      
  if 'lat_bnds' in list(pp.variables):
	  pp = pp.drop_vars('lat_bnds')
      
  if 'axis_nbounds' in list(pp.variables):
    pp = pp.drop_vars('axis_nbounds')
  
  if 'plev_bnds' in list(pp.variables):
    pp = pp.drop_vars('plev_bnds')
      
  ds_out = xr.Dataset(
              {
              "lat": (["latitude"], np.arange(llat, ulat, 0.5)),
              "lon": (["longitude"], np.arange(llon, rlon, 0.5)),
              }
              )    
      # set up the regridding
  try:  
    print('Trying conservative regridding')
    regridder = xe.Regridder(pp, ds_out, "conservative")
  except:
    print('Conservative regridding failed:')
    print('Using bilinear regridding')
    regridder = xe.Regridder(pp, ds_out, 'bilinear')

  ds_in = regridder(pp)

  %rm "$fileName"
  del pp
  
  return ds_in

def sort_precip(rh):
  vx = []
  ppr = rh['pr'].values*86400
  cc = ppr.shape[1]*ppr.shape[2]

  for i in np.arange(np.shape(ppr)[0]):
    v1 = (np.count_nonzero(ppr[i,:,:] >= 1)/cc)*100
    v5 = (np.count_nonzero(ppr[i,:,:] >= 5)/cc)*100
    v10 = (np.count_nonzero(ppr[i,:,:] >= 10)/cc)*100
    vx.append([rh['time'].values[i],np.round(v1,2),np.round(v5,2),np.round(v10,2)])
    #print([rh['time'].values[i],np.round(v1,3),np.round(v5,3),np.round(v10,3)])



  vy = pd.DataFrame(vx,columns=['Date','1mm','5mm','10mm'])
  vy = vy.set_index('Date')
  
  return vy

In [ ]:
writer = pd.ExcelWriter('results.xlsx',mode='a',if_sheet_exists='replace', engine="openpyxl")
for i, ii in enumerate(source):
  for j, jj in enumerate(['ssp2_4_5','ssp5_8_5']):
    print([ii,jj])
    try:
      rh = era5_extractor(jj,ii)
    except:
      continue
    cb = sort_precip(rh)
    cb.to_excel(writer, sheet_name=ii+'_'+jj)

writer.save()

['ACCESS-CM2', 'ssp2_4_5']


2022-08-20 19:36:44,222 INFO Welcome to the CDS
2022-08-20 19:36:44,224 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2022-08-20 19:36:44,389 INFO Request is completed
2022-08-20 19:36:44,392 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data6/adaptor.esgf_wps.retrieve-1661011271.0059571-7024-19-1740a4cb-6aeb-4aae-8316-ed112c7db533.zip to temp.zip (11.7M)
2022-08-20 19:36:46,301 INFO Download rate 6.2M/s


Trying conservative regridding
Conservative regridding failed:
Using bilinear regridding
Overwrite existing file: bilinear_6x13_38x71.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


/usr/local/lib/python3.7/site-packages/xesmf/frontend.py:486: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs
2022-08-20 19:36:56,007 INFO Welcome to the CDS
2022-08-20 19:36:56,009 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6


['ACCESS-CM2', 'ssp5_8_5']


...

2022-08-20 21:38:31,082 INFO Request is queued
2022-08-20 21:38:32,209 INFO Request is running
2022-08-20 21:39:47,135 INFO Request is completed
2022-08-20 21:39:47,138 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data4/adaptor.esgf_wps.retrieve-1661031579.818009-24344-18-6ddcfe94-05b3-42c4-9d4e-74a4e4b09033.zip to temp.zip (7.9M)
2022-08-20 21:39:48,900 INFO Download rate 4.5M/s


Trying conservative regridding
Conservative regridding failed:
Using bilinear regridding
Overwrite existing file: bilinear_4x9_38x71.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


/usr/local/lib/python3.7/site-packages/xesmf/frontend.py:486: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs
2022-08-20 21:39:59,795 INFO Welcome to the CDS
2022-08-20 21:39:59,796 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6


['NorESM2-MM', 'ssp2_4_5']


2022-08-20 21:39:59,933 INFO Request is queued
2022-08-20 21:40:01,060 INFO Request is running


In [ ]:
writer = pd.ExcelWriter('results_historical.xlsx',mode='a',if_sheet_exists='replace', engine="openpyxl")
for i, ii in enumerate(source):
  for j, jj in enumerate(['historical']):
    print([ii,jj])
    try:
      rh = era5_extractor(jj,ii)
    except:
      continue
    cb = sort_precip(rh)
    cb.to_excel(writer, sheet_name=ii+'_'+jj)

writer.save()